In [22]:
import geoengine as ge
import geopandas as gpd
from datetime import datetime

In [23]:
ge.initialize("http://localhost:3030/api", credentials=("admin@localhost", "admin1234"))

In [24]:
monthly_band_data_map = {
    'B02': {'dataset': '6aea7c86-05a6-45fc-ab3f-cd1020a83026', 'upload': '098cdd14-20b6-4a18-a896-01cee17321d5'},
    'B03': {'dataset': '732bcff7-41b2-47a9-9a43-bb5d66feb8fd', 'upload': '37c851a4-a8a7-4af7-9e60-ffefafc48565'},
    'B04': {'dataset': '209ffd4e-d750-4d88-a68a-5c36d739eb75', 'upload': 'bcedc656-7231-45b2-9242-6855e508aea2'},
    'B08': {'dataset': 'ba469628-1c48-4172-af01-6bc460690600', 'upload': '5c376f85-4f55-414e-928d-319730320d7a'},
    'NDVI': {'dataset': 'a5fbef96-0de3-498e-9e38-ddaf3a9fe47a', 'upload': '6bb9955f-3683-4de6-97da-3dba9dfe493e'}
}
monthly_band_data_map

{'B02': {'dataset': '6aea7c86-05a6-45fc-ab3f-cd1020a83026',
  'upload': '098cdd14-20b6-4a18-a896-01cee17321d5'},
 'B03': {'dataset': '732bcff7-41b2-47a9-9a43-bb5d66feb8fd',
  'upload': '37c851a4-a8a7-4af7-9e60-ffefafc48565'},
 'B04': {'dataset': '209ffd4e-d750-4d88-a68a-5c36d739eb75',
  'upload': 'bcedc656-7231-45b2-9242-6855e508aea2'},
 'B08': {'dataset': 'ba469628-1c48-4172-af01-6bc460690600',
  'upload': '5c376f85-4f55-414e-928d-319730320d7a'},
 'NDVI': {'dataset': 'a5fbef96-0de3-498e-9e38-ddaf3a9fe47a',
  'upload': '6bb9955f-3683-4de6-97da-3dba9dfe493e'}}

In [25]:
monthly_band_data_id_map = {
    name: ge.api.InternalDataId(
        type="internal",
        datasetId=id['dataset']
    ) for name, id in monthly_band_data_map.items()
} 
monthly_band_data_id_map

{'B02': {'type': 'internal',
  'datasetId': '6aea7c86-05a6-45fc-ab3f-cd1020a83026'},
 'B03': {'type': 'internal',
  'datasetId': '732bcff7-41b2-47a9-9a43-bb5d66feb8fd'},
 'B04': {'type': 'internal',
  'datasetId': '209ffd4e-d750-4d88-a68a-5c36d739eb75'},
 'B08': {'type': 'internal',
  'datasetId': 'ba469628-1c48-4172-af01-6bc460690600'},
 'NDVI': {'type': 'internal',
  'datasetId': 'a5fbef96-0de3-498e-9e38-ddaf3a9fe47a'}}

In [26]:
bounds_array = [421395,  5681078, 476201, 5727833]
xmin = bounds_array[0]
ymin = bounds_array[1]
xmax = bounds_array[2]
ymax = bounds_array[3]

(xmin, ymin, xmax, ymax)

(421395, 5681078, 476201, 5727833)

In [27]:
start_dt = datetime(2021, 10, 15, 0, 0, 0)
end_dt = datetime(2021, 10, 15, 0, 0, 0)

box_size = 1024

box_center_x = 0.5 * (xmin + xmax)
box_center_y = 0.5 * (ymin + ymax)

box_x_min = box_center_x - box_size / 2
box_x_max = box_x_min + box_size
box_y_min = box_center_y - box_size / 2
box_y_max = box_y_min + box_size

query_rect = ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=box_x_min,
            ymin=box_y_min,
            xmax=box_x_max,
            ymax=box_y_max,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            10.0,
            10.0,
        ),
        srs="EPSG:32632",
)

query_rect

QueryRectangle( 
    BoundingBox2D(xmin=448542.0, ymin=5704199.5, xmax=449054.0, ymax=5704711.5)
    TimeInterval(start=2021-10-15 00:00:00+00:00, end=2021-10-15 00:00:00+00:00)
    SpatialResolution(x=10.0, y=10.0)
    srs=EPSG:32632 
)

In [28]:
band_workflows_a = {b: ge.unstable.workflow_operators.GdalSource(x) for b, x in monthly_band_data_id_map.items()}

for w in band_workflows_a.values():
    workflow_id = ge.register_workflow(w.to_workflow_dict())
    print(workflow_id)



8e3f4869-0d7d-53b4-b1e8-17124433d159
4bfae55d-47fa-5bf8-888e-6a656268bc2a
72aa13c7-52a2-5554-aaf3-9193f23aae6d
e5c3d856-3b90-53c9-b382-264070b0a501
7616e304-8a5b-55b5-9ee9-752909c34a83


In [29]:

workflow = ge.register_workflow(band_workflows_a['B02'].to_workflow_dict())

workflow.get_result_descriptor()


Data type:         U16
Spatial Reference: EPSG:32632
Measurement:       unitless

In [30]:
async for tile_stac in workflow.raster_stream(query_rect):
    print(len(tile_stac))
    print(tile_stac[0].shape)

Received 526606 bytes
Processing 526606 bytes
read_arrow_ipc
read_arrow_ipc 2
1
data: uint16
-- schema metadata --
xSize: '512'
geoTransform: '{"originCoordinate":{"x":445440.0,"y":5708800.0},"xPixelSi' + 28
ySize: '512'
time: '{"start":1633046400000,"end":1635724800000}'
spatialReference: 'EPSG:32632'
ReadStats(num_messages=1, num_record_batches=0, num_dictionary_batches=0, num_dictionary_deltas=0, num_replaced_dictionaries=0)


ArrowInvalid: Negative buffer resize: -1